In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16").to("cuda")

import gradio as gr

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

def generate(prompt):
  width = closestNumber(512, 8)
  height = closestNumber(512, 8)
  image = pipe(prompt, num_inference_steps=1, guidance_scale=0.0, width=width, height=height).images[0]
  image.save('/content/image.jpg')
  return image.resize((512, 512))

with gr.Blocks(title=f"sdxl-turbo", css=".gradio-container {max-width: 544px !important}", analytics_enabled=False) as demo:
    with gr.Row():
      with gr.Column():
          textbox = gr.Textbox(show_label=False, value="a close-up picture of a fluffy cat")
          button = gr.Button()
    with gr.Row(variant="default"):
        output_image = gr.Image(
            show_label=False,
            type="pil",
            interactive=False,
            height=512,
            width=512,
            elem_id="output_image",
        )

    button.click(fn=generate, inputs=[textbox], outputs=[output_image], show_progress=False)

demo.queue().launch(inline=False, share=False, debug=True, server_name='0.0.0.0', server_port=1002)

In [ ]:
import http.server, socketserver, os, json
import requests, base64, threading
from pymongo import MongoClient

mongodb = 'mongodb+srv://'
api = 'http://127.0.0.1:1002'
token = ''
job = 'sdxl'

def loop():
  client = MongoClient(mongodb)
  db = client['web']
  collection_job = db['job']
  collection_detail = db['detail']

  while True:
    waiting_documents = collection_job.find({"status": "WAITING"})
    for waiting_document in waiting_documents:
        detail = waiting_document['discord']
        # print(waiting_document)
        # print(detail.id)
        discord = collection_detail.find_one({"_id": detail.id})['discord']
        total = collection_detail.find_one({"_id": detail.id})['total']
        server = waiting_document['type']
        if(server==job):
            amount = waiting_document['amount']
            command = waiting_document['command']
            source_channel = waiting_document['source_channel']
            source_id = waiting_document['source_id']
            collection_job.update_one({"_id": waiting_document['_id']}, {"$set": {"status": "WORKING"}})
            try:
                from gradio_client import Client
                client = Client(api, verbose=False)
                result = client.predict(command, fn_index=0)
                # print(result)
                files = {f"image.png": open(result, "rb").read()}
                payload = {"content": f"{command} <@{source_id}>"}
                try:
                    responseD = requests.post(f"https://discord.com/api/v9/channels/{source_channel}/messages", data=payload, headers={"authorization": f"Bot {token}"}, files=files)
                    responseD.raise_for_status()  # Raise an exception for 4xx and 5xx errors
                    collection_job.update_one({"_id": waiting_document['_id']}, {"$set": {"status": "DONE"}})
                    collection_job.update_one({"_id": waiting_document['_id']}, {"$set": {"result": responseD.json()['attachments'][0]['url']}})
                    total = int(total) - int(amount)
                    collection_detail.update_one({"_id": detail.id}, {"$set": {"total": total}})
                    # print(responseD.text)
                    # print(responseD.json()['attachments'][0]['url'])
                except requests.exceptions.RequestException as e:
                    print(f"D An error occurred: {e}")
                except Exception as e:
                    print(f"D An unexpected error occurred: {e}")
            except requests.exceptions.RequestException as e:
                print(f"F An error occurred: {e}")
            except Exception as e:
                print(f"F An unexpected error occurred: {e}")

class MyHandler(http.server.SimpleHTTPRequestHandler):
    def translate_path(self, path):
        path = super().translate_path(path)
        if path.endswith('.py'):
            self.send_error(404, "File not found")
            return None
        return path
      
PORT = 1003
Handler = MyHandler
Handler.extensions_map.update({
    '.html': 'text/html',
})

with socketserver.TCPServer(("", PORT), Handler) as httpd:
    thread = threading.Thread(target=loop)
    thread.start()
    print(f"Server running on port {PORT}")
    httpd.serve_forever()